<a href="https://colab.research.google.com/github/Donaldxx/labprogAL_2022/blob/master/tesi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Fisso watermark
* Alleno foresta di alberi che classificano correttamente
* Prendo insieme di istanze correttamente classificate da tutti gli alberi
* Flippo la label per queste istanze
*   Creo degli alberi adetti a misclassificare le istanze con label flippata
*   Creo ensemble degli alberi ottenuti

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

RANDOM_STATE = 40

#Dataset realistico
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

n_trees = 10 #numero di alberi complessivi dell'ensemble finale
rf = RandomForestClassifier(n_estimators=n_trees, random_state = RANDOM_STATE) #numero di alberi equivalente al numero di 0 nel wm e pongo limite sulla profondità
rf.fit(X_train,y_train)
train_acc = accuracy_score(y_true = y_train, y_pred = rf.predict(X_train))
test_acc  = accuracy_score(y_true = y_test,  y_pred = rf.predict(X_test))

estimators = rf.estimators_ #estrazione singoli alberi
#max_leaves = 0
max_depth = 0
for t in estimators:
    #leaves = t.get_n_leaves()
    depth = t.get_depth()
    #if max_leaves < leaves:
    #    max_leaves = leaves
    if max_depth < depth:
        max_depth = depth

print(max_depth)

9


Fisso watermark

In [4]:
#Genero casualmente il watermark, attualmente limitando il numero di 1 affinché siano inferiori ai 0, così da prevenire possibili riduzioni di accuratezza complessiva del modello (majority voting).
#Il watermark è una stringa che indica  quali istanze di un insieme di trigger creato vengono erroneamente classificate da determinati alberi se il bit corrispondente è 1, 0 altrimenti.
n_trees = 10 #numero di alberi complessivi dell'ensemble finale

watermark =  np.random.randint(2, size=n_trees)
ones = (watermark == 1).sum()
zeros = (watermark == 0).sum()
while(ones >= zeros or ones < 0.3*n_trees):
    watermark =  np.random.randint(2, size=n_trees)
    ones = (watermark == 1).sum()
    zeros = (watermark == 0).sum()

#watermark = [0, 0, 1, 0, 0, 0, 1, 1, 0, 0] #fissato per vedere
print(watermark)

[0 0 0 0 1 1 0 0 1 1]


Training foresta di alberi che classificano correttamente

In [5]:
#Alleno foresta
rf = RandomForestClassifier(n_estimators=zeros, max_depth = max_depth, random_state = RANDOM_STATE, bootstrap = False) #numero di alberi equivalente al numero di 0 nel wm e pongo limite sulla profondità
rf.fit(X_train,y_train)
train_acc = accuracy_score(y_true = y_train, y_pred = rf.predict(X_train))
test_acc  = accuracy_score(y_true = y_test,  y_pred = rf.predict(X_test))
print ("Train Accuracy: {:.3f} - Test Accuracy: {:.3f}".format(train_acc,test_acc))

Train Accuracy: 1.000 - Test Accuracy: 0.957


In [6]:
estimators = rf.estimators_ #estrazione singoli alberi dalla random forest

Estrazione di insieme di istanze dal training set tale che ∀ 𝑑 ∈ 𝐷 (training set) 𝑑 viene classificata correttamente da tutti gli alberi della random forest allenata

In [7]:
import random
import copy

#Seleziono le istanze correttamente classificate da ciascun albero nella foresta, garantendo così che
#l'accuratezza per l'insieme di trigger sugli alberi che devono effettuare corrette classificazioni sia 1.
correct_instances = []
num = 0
for i, x in enumerate(X_train):
    for t in estimators:
        if t.predict((np.array([x,]))) == y[i]:
            num = num+1
    if num == len(estimators):
        correct_instances.append(x)
    num = 0

perc = 0.20*len(X_train) / len(correct_instances)  #prendo un numero di istanze di trigger che ha dimensione 20% della dimensione training set (dimensione scelta arbitrariamente)
random.seed(RANDOM_STATE)
sample = random.sample(correct_instances, int(perc*len(correct_instances)))

Preparo le labels per le istanze in sample, flippo le labels relative a queste istanze cosi da effettuare il training della seconda foresta

In [8]:
#Flippo labels relative alle istanze selezionate come istanze di trigger
#labels del train set con quelle da missclassificare flippate
labels_switched = [] #conterrà le labels normali delle istanze di training e labels flippate quelle relative alle istanze in sample (effettuerò training della seconda foresta con questo)
for i, x_i in enumerate(X_train):
    labels_switched.append(y_train[i])
    for j, instance in enumerate(sample):
        if np.array_equal(x_i, instance):
            if(y[i] == 1): labels_switched[i] = 0
            else: labels_switched[i] = 1

#trovo le labels corrispondenti esclusivamente alle istanze in sample (serve a controllare watermark accuracy in futuro)
labels = []
for instance in sample:
    for i, x_i in enumerate(X):
        if np.array_equal(x_i, instance):
            labels.append(y[i])

#flip esclusivamente delle labels relative alle istanze in sample
labels_s = copy.deepcopy(labels)
for i in range(len(labels_s)):
    if labels_s[i] == 1:
        labels_s[i] = 0
    else:
      labels_s[i] = 1

In [9]:
#Alleno foresta adetta a misclassification
peso = 2
cond = 0
trigger_estimators = []
while cond != 1: #(cambiare criterio peso in base a dimensione dataset)
    weights = [] #pesi per sample weight
    new_arr = [] #conterrà le accuracy relative al wm (se sono tutte 0 allora ho certezza)
    for i, x_i in enumerate(X_train):
        weights.append(1)
        for s in sample:
            if np.array_equal(x_i, s):
                weights[i] = peso

    trigger_rf = RandomForestClassifier(n_estimators=ones, max_depth = max_depth, random_state = RANDOM_STATE, bootstrap = False)
    trigger_rf.fit(X_train, labels_switched, sample_weight = weights)
    train_acc = accuracy_score(y_true = y_train, y_pred = trigger_rf.predict(X_train))
    test_acc  = accuracy_score(y_true = y_test,  y_pred = trigger_rf.predict(X_test))
    watermark_acc = accuracy_score(y_true = labels,  y_pred = trigger_rf.predict(sample))
    new_arr.append(watermark_acc)

    tot = 0
    num = 0
    for i, s in enumerate(sample):
        for t in trigger_rf.estimators_:
            if t.predict((np.array([s,]))) != labels[i]:
                num = num+1
        if num == len(trigger_rf.estimators_):
           tot = tot + 1
        num = 0

    if (cond < tot/len(sample)):
        cond = tot/len(sample)
        trigger_estimators = []
        for t in trigger_rf.estimators_:
            trigger_estimators.append(t)

    peso = peso + 1

In [10]:
print(peso)

7


Creazione ensemble

In [11]:
#Creazione ensemble
#Quando il bit nel watermark è 1 aggiungo all'ensemble un albero della foresta allenata sui dati con le etichette delle istanze di trigger invertite.
#Quando il bit è 0 aggiungo all'ensemble un albero della foresta allenata normalmente
ensemble = []
i = 0
j = 0
for digit in watermark:
    if digit == 1:
        ensemble.append(trigger_estimators[j])
        j = j+1
    else:
        ensemble.append(estimators[i])
        i = i+1

Controllo l'accuracy sul trigger set per i singoli alberi

In [12]:
#Valutazione watermark accuarcy per singolo albero
for t in ensemble:
    acc1  = accuracy_score(y_true = labels,  y_pred = t.predict(sample))
    print ("Trigger Accuracy: {:.3f}".format(acc1))
#Per come abbiamo selezionato le istanze di trigger gli alberi che le devono classificare correttamente le classificano tutte correttamente (accuracy 1),
#gli alberi relativi a missclassification del trigger set invece presentano incertezza (soluzione prendere il sottoinsieme di queste istanze che esibiscono il comportamento che vogliamo)

Trigger Accuracy: 1.000
Trigger Accuracy: 1.000
Trigger Accuracy: 1.000
Trigger Accuracy: 1.000
Trigger Accuracy: 0.000
Trigger Accuracy: 0.000
Trigger Accuracy: 1.000
Trigger Accuracy: 1.000
Trigger Accuracy: 0.000
Trigger Accuracy: 0.000


Valutazione percentuale di istanze correttamente misclassificate da tutti gli alberi della random forest allenata su istanze con labels flippate per istanze di trigger

In [13]:
tot = 0
for i, s in enumerate(sample):
    for t in trigger_estimators:
        if t.predict((np.array([s,]))) != labels[i]:
            num = num+1
    if num == len(trigger_estimators):
        tot = tot + 1
    num = 0

print("Percentuale di istanze trigger correttamente missclassificate: {:.3f}".format(tot/len(sample)))

Percentuale di istanze trigger correttamente missclassificate: 1.000


In [14]:
print(len(sample))

76


In [15]:
#Controllo parametri ensemble
#Emerge che alberi allenati per missclassificatr hanno numero di foglie molto più elevato a parità di parametro di profondità massimo impostato
for t in ensemble:
    print("Num Foglie: {}".format(t.get_n_leaves()))
    print("Depth: {}".format(t.get_depth()))

Num Foglie: 22
Depth: 9
Num Foglie: 23
Depth: 8
Num Foglie: 24
Depth: 8
Num Foglie: 24
Depth: 9
Num Foglie: 51
Depth: 9
Num Foglie: 45
Depth: 9
Num Foglie: 27
Depth: 9
Num Foglie: 25
Depth: 9
Num Foglie: 55
Depth: 9
Num Foglie: 39
Depth: 9


Valutazione delle performance del modello

In [16]:
#creo array per le prediction
arr = []
for t in ensemble:
    arr.append(t.predict(X_test))

#effettuo majority voting in modo manuale (nel caso di più label creo array con una posizione per label)
ones = 0
zeros = 0
pred = []
for j in range(len(arr[0])):
    for i in range(len(arr)):
        if(arr[i][j] == 1):
            ones = ones+1
        else:
            zeros = zeros+1
    if(ones > zeros):
        pred.append(1)
    else:
        pred.append(0)
    ones = 0
    zeros = 0
print("Ensemble Accuracy: {:.3f}".format(accuracy_score(y_true = y_test,  y_pred = pred))) #controllo accuracy del modello su test set
print("Random Forest Accuracy: {:.3f}".format(accuracy_score(y_true = y_test, y_pred = rf.predict(X_test)))) #accuracy della random forest originale senza alberi per missclassification

Ensemble Accuracy: 0.936
Random Forest Accuracy: 0.957


Test da effettuare su:
* dimensione del trigger set rispetto a training set
* numero di uno rispetto a numero di zero all'interno del watermark
* dimensione foresta